In [1]:
! pip install -U scikit-learn
! pip install numpy==1.17.0
! pip install nltk

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.2
    Uninstalling scikit-learn-1.1.2:
      Successfully uninstalled scikit-learn-1.1.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'f:\Program Files own\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ham_url = './data/ham/ham'
spam_url = './data/spam/spam'

ham_lists = os.listdir(ham_url)
spam_lists = os.listdir(spam_url)
# spam 1
# ham 0
ham_frame = pd.DataFrame(data={'email':'','label':np.zeros(len(ham_lists))})
spam_frame = pd.DataFrame(data={'email':'','label':np.ones(len(spam_lists))})

In [3]:
#ham data
for i in range(len(ham_lists)):
    file = open('./data/ham/ham/'+ham_lists[i],'rb')
    content = file.read()
    file.close()
    ham_frame['email'][i] = content
#spam data
for i in range(len(spam_lists)):
    file = open('./data/spam/spam/'+spam_lists[i],'rb')
    content = file.read()
    file.close()
    spam_frame['email'][i] = content
    
emails_frame = pd.concat([ham_frame,spam_frame],axis = 0, ignore_index=True)

C:\Users\hjr\AppData\Local\Temp\ipykernel_20812\2802281897.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ham_frame['email'][i] = content
C:\Users\hjr\AppData\Local\Temp\ipykernel_20812\2802281897.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spam_frame['email'][i] = content


In [4]:
# preprocess
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
def stopword(text):
    str=''
    wordList = text.split()
    for word in wordList:
        if word not in stop_words:
            str = str+word+' '
    return str

In [5]:
def getStemmer(text):
    str=''
    stemmer = nltk.porter.PorterStemmer()
    wordList = text.split()
    for word in wordList:
        word = stemmer.stem(word)
        str = str+word+' '
    return str

In [6]:
import re
import string
translator = re.compile('[%s]' % re.escape(string.punctuation))
def rexEmail(email):
    text = translator.sub(' ', email)
    text = re.sub('\n+','',text)
    text = re.sub('\t+','',text)
    text = re.sub(r'[0-9]+',' ',text)
    text = text.lower()
    return text

In [7]:

from email.parser import BytesParser
from email.policy import default


emails = []
emails_frame['content'] = None
for i in range(len(emails_frame)):
    email =  BytesParser(policy=default).parsebytes(emails_frame['email'][i])
    if not email.is_multipart():
        data = email.get_payload()
        emails_frame['content'][i] = getStemmer(stopword(rexEmail(str(data))))

C:\Users\hjr\AppData\Local\Temp\ipykernel_20812\3484896855.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emails_frame['content'][i] = getStemmer(stopword(rexEmail(str(data))))


In [8]:
filtered_emails_frame = emails_frame.dropna(axis=0)
filtered_emails_frame = filtered_emails_frame.reset_index(drop=True)
filtered_emails_frame

,email,label,content
0,b'From exmh-workers-admin@redhat.com Thu Aug ...,0.0,date wed aug chri garrigu cwg date fa deepeddi...
1,b'Return-Path: <Online#3.19578.34-UgGTgZFN19NA...,0.0,html head titl cabl compani crack wi fi titl h...
2,b'Return-Path: <Online#3.19584.83-p1SYlJ1blFvQ...,0.0,html head titl shopper newslett alert titl hea...
3,b'From Steve_Burt@cursor-system.com Thu Aug 2...,0.0,martin post tasso papadopoulo greek sculptor b...
4,b'Return-Path: <Online#3.19586.b5-9w0blztbvHPd...,0.0,cnet download dispatch mac edit juli vol use m...
...,...,...,...
3058,b'From 2002biz2biz2513@Flashmail.com Mon Oct ...,1.0,stumbl greatest way market centuri undoubtedli...
3059,b'From biz2biz2446@Flashmail.com Mon Oct 7 2...,1.0,stumbl greatest way market centuri undoubtedli...
3060,b'From bounce2@u-answer.com Tue Oct 8 11:02:...,1.0,html head meta http equiv content languag cont...
3061,b'From beautyinfufuxxxmeb13mxy@aol.com Tue Oc...,1.0,html bodi tr valign top td height bgcolor ffff...


In [107]:
filtered_emails_frame.to_csv('Preprocess_emails.csv',index=False)